In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
con = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/모델/content_surprise/csv/category/c_score_rate.csv')
# con.columns = ['subsr', 'program', 'category','vod_id', 't_score', 'ratings']
con

,subsr,rename,Category,vod_id,Affinity,ratings
0,59879000,나쁜엄마,TV프로그램,3403,0.405465,1.0
1,59879000,나의해방일지,TV프로그램,1859,0.405465,1.0
2,59879000,낭만닥터김사부2,TV프로그램,1168,0.405465,1.0
3,59879000,소방서옆경찰서,TV프로그램,2092,1.504077,3.0
4,59879000,슬기로운의사생활시즌2,TV프로그램,2891,0.405465,1.0
...,...,...,...,...,...,...
7700,67160000,심야괴담회,TV프로그램,143,0.637052,1.0
7701,67160000,콘크리트유토피아,영화,2630,0.659090,3.0
7702,67161000,스파이코드명포춘,영화,3824,1.705884,5.0
7703,67164000,미션임파서블:데드레코닝PARTONE,영화,2867,1.083268,1.0


In [7]:
vod_list = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/모델/content_surprise/csv/vod_list_add10.csv', index_col=0)
vod_id = vod_list[['rename', 'Category', 'vod_id']]
vod_id.columns = ['rename', 'category', 'vod_id']
vod_id

,rename,category,vod_id
0,와이낫크루뜻밖의여행,TV프로그램,0
1,그레이트뷰티,영화,1
2,해안선,영화,2
3,아는형님,TV프로그램,3
4,베놈,영화,4
...,...,...,...
5019,일루셔니스트,영화,5019
5020,"파리,13구",영화,5020
5021,미쓰백,영화,5021
5022,그녀는거짓말을너무사랑해,영화,5022


In [8]:
con1 = con.copy()
con1

,subsr,rename,Category,vod_id,Affinity,ratings
0,59879000,나쁜엄마,TV프로그램,3403,0.405465,1.0
1,59879000,나의해방일지,TV프로그램,1859,0.405465,1.0
2,59879000,낭만닥터김사부2,TV프로그램,1168,0.405465,1.0
3,59879000,소방서옆경찰서,TV프로그램,2092,1.504077,3.0
4,59879000,슬기로운의사생활시즌2,TV프로그램,2891,0.405465,1.0
...,...,...,...,...,...,...
7700,67160000,심야괴담회,TV프로그램,143,0.637052,1.0
7701,67160000,콘크리트유토피아,영화,2630,0.659090,3.0
7702,67161000,스파이코드명포춘,영화,3824,1.705884,5.0
7703,67164000,미션임파서블:데드레코닝PARTONE,영화,2867,1.083268,1.0


In [9]:
con_score = con1[['subsr', 'vod_id', 'ratings']]
con_score

,subsr,vod_id,ratings
0,59879000,3403,1.0
1,59879000,1859,1.0
2,59879000,1168,1.0
3,59879000,2092,3.0
4,59879000,2891,1.0
...,...,...,...
7700,67160000,143,1.0
7701,67160000,2630,3.0
7702,67161000,3824,5.0
7703,67164000,2867,1.0


In [10]:
! pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163754 sha256=2981efe51c372e206e3b7d01011b4e5e1964514c4c6cc0be4544c4a897d85a96
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [12]:
# 필요한 Surprise 알고리즘 불러오기
from surprise import SVD, BaselineOnly, SVDpp, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline
from surprise import accuracy
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# surprise 데이터 형식으로 변환
def convert_traintest_dataframe_forsurprise(training_dataframe):
    reader = Reader(rating_scale=(0, 1)) # 범위가 0~1 인 경우
    trainset = Dataset.load_from_df(training_dataframe[['subsr', 'vod_id', 'ratings']], reader)
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    return trainset

trainset = convert_traintest_dataframe_forsurprise(con_score)

### KNNBaseline - user-based similarity
- **cosine (0.0004950495049504951)**
- pearson_baseline (0)
- msd (0.0004950495049504951)

In [36]:
sim_options = {'name': 'msd',
               'user_based': True  # user-based similarity(사용자 기반 협업 필터)
               }
# 최근접 이웃의 수를 6으로 지정, 훈련할 때 사용되는 난수 발생 시드 지정
knnbaseline = KNNBaseline(k = 6, sim_options=sim_options, random_state = 42)
knnbaseline.fit(trainset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [37]:
user_id = sorted(con_score.subsr.unique())
vod_id = sorted(con_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(knnbaseline.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
# result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,real,predict
0,59879000,0,None,1.0
1,59879000,1,None,1.0
2,59879000,3,None,1.0
3,59879000,5,None,1.0
4,59879000,6,None,1.0
...,...,...,...,...
2659843,67164000,4708,None,1.0
2659844,67164000,4710,None,1.0
2659845,67164000,4711,None,1.0
2659846,67164000,4712,None,1.0


In [38]:
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,1.0
1,59879000,1,1.0
2,59879000,3,1.0
3,59879000,5,1.0
4,59879000,6,1.0
...,...,...,...
2659843,67164000,4708,1.0
2659844,67164000,4710,1.0
2659845,67164000,4711,1.0
2659846,67164000,4712,1.0


In [39]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,0,3010,3014,3015,3016,3017,3019,3021,3022,...,3034,3011,3004,2818,3002,2968,2970,2973,2974,2976
1,59882000,0,2978,2963,2964,2968,2970,2973,2974,2976,...,2962,2959,3002,2939,2923,2924,2927,2928,2929,2931
2,59886000,0,3010,3014,3015,3016,3017,3019,3021,3022,...,3034,3011,3004,2818,3002,2968,2970,2973,2974,2976
3,59890000,0,3010,3014,3015,3016,3017,3019,3021,3022,...,3034,3011,3004,3126,3002,2968,2970,2973,2974,2976
4,59892000,0,2970,2957,2958,2959,2960,2962,2963,2964,...,2954,2952,2986,2929,2911,2913,2918,2922,2924,2926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,67149000,0,2959,2945,2947,2949,2952,2953,2954,2957,...,2943,2939,2892,2918,2895,2897,2900,2903,2906,2908
917,67154000,0,3010,3014,3015,3016,3017,3019,3021,3022,...,3034,3011,3004,3126,3002,2968,2970,2973,2974,2976
918,67160000,0,3004,3011,3014,3015,3016,3017,3019,3021,...,3033,3010,3002,2817,3001,2964,2968,2970,2973,2974
919,67161000,0,3010,3014,3015,3016,3017,3019,3021,3022,...,3034,3011,3004,3448,3002,2968,2970,2973,2974,2976


In [40]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59879000,"[0, 3010, 3014, 3015, 3016, 3017, 3019, 3021, ..."
1,59882000,"[0, 2978, 2963, 2964, 2968, 2970, 2973, 2974, ..."
2,59886000,"[0, 3010, 3014, 3015, 3016, 3017, 3019, 3021, ..."
3,59890000,"[0, 3010, 3014, 3015, 3016, 3017, 3019, 3021, ..."
4,59892000,"[0, 2970, 2957, 2958, 2959, 2960, 2962, 2963, ..."
...,...,...
916,67149000,"[0, 2959, 2945, 2947, 2949, 2952, 2953, 2954, ..."
917,67154000,"[0, 3010, 3014, 3015, 3016, 3017, 3019, 3021, ..."
918,67160000,"[0, 3004, 3011, 3014, 3015, 3016, 3017, 3019, ..."
919,67161000,"[0, 3010, 3014, 3015, 3016, 3017, 3019, 3021, ..."


In [41]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59879000,"[0.0, 3010.0, 3014.0, 3015.0, 3016.0, 3017.0, ..."
1,59882000,"[0.0, 2978.0, 2963.0, 2964.0, 2968.0, 2970.0, ..."
2,59886000,"[0.0, 3010.0, 3014.0, 3015.0, 3016.0, 3017.0, ..."
3,59890000,"[0.0, 3010.0, 3014.0, 3015.0, 3016.0, 3017.0, ..."
4,59892000,"[0.0, 2970.0, 2957.0, 2958.0, 2959.0, 2960.0, ..."
...,...,...
916,67149000,"[0.0, 2959.0, 2945.0, 2947.0, 2949.0, 2952.0, ..."
917,67154000,"[0.0, 3010.0, 3014.0, 3015.0, 3016.0, 3017.0, ..."
918,67160000,"[0.0, 3004.0, 3011.0, 3014.0, 3015.0, 3016.0, ..."
919,67161000,"[0.0, 3010.0, 3014.0, 3015.0, 3016.0, 3017.0, ..."


In [28]:
testdata = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/모델/content_surprise/csv/watched_vod_10.csv', index_col=0)
testdata = testdata.groupby('subsr')['vod_id'].unique().reset_index()
testdata

,subsr,vod_id
0,59900000,"[1278, 1885]"
1,59930000,[2098]
2,59933000,[296]
3,60050000,"[200, 2415, 2869, 2546, 213, 2291, 2610, 4880]"
4,60067000,"[1785, 887, 1347, 4773, 4782, 4759, 992, 3806,..."
...,...,...
215,67133000,[296]
216,67164000,"[4956, 1434, 4995, 2058]"
217,67170000,"[3464, 948, 3900, 675]"
218,67202000,[1028]


In [42]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict_1)

0.0004950495049504951

### KNNBaseline - item-based similarity
- cosine (0)
- pearson_baseline (0)
- msd (0)

In [50]:
sim_options = {'name': 'msd',
               'user_based': False  # user-based similarity(사용자 기반 협업 필터)
               }
# 최근접 이웃의 수를 6으로 지정, 훈련할 때 사용되는 난수 발생 시드 지정
knnbaseline = KNNBaseline(k = 6, sim_options=sim_options, random_state = 42)
knnbaseline.fit(trainset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [51]:
user_id = sorted(con_score.subsr.unique())
vod_id = sorted(con_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(knnbaseline.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
# result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,real,predict
0,59879000,0,None,1.0
1,59879000,1,None,1.0
2,59879000,3,None,1.0
3,59879000,5,None,1.0
4,59879000,6,None,1.0
...,...,...,...,...
2659843,67164000,4708,None,1.0
2659844,67164000,4710,None,1.0
2659845,67164000,4711,None,1.0
2659846,67164000,4712,None,1.0


In [52]:
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,1.0
1,59879000,1,1.0
2,59879000,3,1.0
3,59879000,5,1.0
4,59879000,6,1.0
...,...,...,...
2659843,67164000,4708,1.0
2659844,67164000,4710,1.0
2659845,67164000,4711,1.0
2659846,67164000,4712,1.0


In [53]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,0,3010,3014,3015,3016,3017,3019,3021,3022,...,3034,3011,3004,2818,3002,2968,2970,2973,2974,2976
1,59882000,0,3426,2964,2968,2970,2973,2976,2977,2980,...,3002,2963,2962,2960,2941,2926,2928,2929,2930,2931
2,59886000,0,2833,2818,2819,2824,2826,2827,2831,2832,...,2769,2784,2773,2778,2779,2780,2781,2782,2783,2787
3,59890000,0,3010,3014,3015,3016,3017,3019,3021,3022,...,3034,3011,3004,3126,3002,2968,2970,2973,2974,2976
4,59892000,2181,2958,2941,2943,2945,2947,2949,2953,2954,...,2939,2935,2883,2900,2885,2887,2888,2890,2891,2892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,67149000,4713,2560,2546,2549,2552,2553,2555,2556,2557,...,2539,2537,2512,2513,2515,2517,4285,2520,2522,2523
917,67154000,0,3010,3014,3015,3016,3017,3019,3021,3022,...,3034,3011,3004,3126,3002,2968,2970,2973,2974,2976
918,67160000,0,2787,2778,2779,2780,2781,2782,2783,2784,...,2768,2764,2718,2720,2722,2723,2732,2733,2734,2738
919,67161000,0,3010,3014,3015,3016,3017,3019,3021,3022,...,3034,3011,3004,3126,3002,2968,2970,2973,2974,2976


In [54]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59879000,"[0, 3010, 3014, 3015, 3016, 3017, 3019, 3021, ..."
1,59882000,"[0, 3426, 2964, 2968, 2970, 2973, 2976, 2977, ..."
2,59886000,"[0, 2833, 2818, 2819, 2824, 2826, 2827, 2831, ..."
3,59890000,"[0, 3010, 3014, 3015, 3016, 3017, 3019, 3021, ..."
4,59892000,"[2181, 2958, 2941, 2943, 2945, 2947, 2949, 295..."
...,...,...
916,67149000,"[4713, 2560, 2546, 2549, 2552, 2553, 2555, 255..."
917,67154000,"[0, 3010, 3014, 3015, 3016, 3017, 3019, 3021, ..."
918,67160000,"[0, 2787, 2778, 2779, 2780, 2781, 2782, 2783, ..."
919,67161000,"[0, 3010, 3014, 3015, 3016, 3017, 3019, 3021, ..."


In [55]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59879000,"[0.0, 3010.0, 3014.0, 3015.0, 3016.0, 3017.0, ..."
1,59882000,"[0.0, 3426.0, 2964.0, 2968.0, 2970.0, 2973.0, ..."
2,59886000,"[0.0, 2833.0, 2818.0, 2819.0, 2824.0, 2826.0, ..."
3,59890000,"[0.0, 3010.0, 3014.0, 3015.0, 3016.0, 3017.0, ..."
4,59892000,"[2181.0, 2958.0, 2941.0, 2943.0, 2945.0, 2947...."
...,...,...
916,67149000,"[4713.0, 2560.0, 2546.0, 2549.0, 2552.0, 2553...."
917,67154000,"[0.0, 3010.0, 3014.0, 3015.0, 3016.0, 3017.0, ..."
918,67160000,"[0.0, 2787.0, 2778.0, 2779.0, 2780.0, 2781.0, ..."
919,67161000,"[0.0, 3010.0, 3014.0, 3015.0, 3016.0, 3017.0, ..."


In [ ]:
testdata = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/모델/content_surprise/csv/watched_vod_10.csv', index_col=0)
testdata = testdata.groupby('subsr')['vod_id'].unique().reset_index()
testdata

,subsr,vod_id
0,59900000,"[1278, 1885]"
1,59930000,[2098]
2,59933000,[296]
3,60050000,"[200, 2415, 2869, 2546, 213, 2291, 2610, 4880]"
4,60067000,"[1785, 887, 1347, 4773, 4782, 4759, 992, 3806,..."
...,...,...
215,67133000,[296]
216,67164000,"[4956, 1434, 4995, 2058]"
217,67170000,"[3464, 948, 3900, 675]"
218,67202000,[1028]


In [56]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict_1)

0.0

### SVD
- SVD basic (0)
- n_epochs= 95, lr_all= 0.005, reg_all= 0.06, random_state= 42 (0.012307692307692311)

In [ ]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': np.arange(150, 1200, 5), 'lr_all': [0.01, 0.05],
              'reg_all': [0.05, 0.09], 'random_state' : [42]}
              # lr_all – The learning rate for all parameters. Default is 0.005.
              # reg_all – The regularization term for all parameters. Default is 0.02.

gs = GridSearchCV(SVD, param_grid, measures=['mae'], cv=5)

reader = Reader(rating_scale=(0,1))
train_set = Dataset.load_from_df(con_score[['subsr', 'vod_id', 'click_score']], reader)

gs.fit(train_set)
gs.best_params

In [58]:
svd = SVD()
svd.fit(trainset)

In [59]:
user_id = sorted(con_score.subsr.unique())
vod_id = sorted(con_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(svd.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,1.0
1,59879000,1,1.0
2,59879000,3,1.0
3,59879000,5,1.0
4,59879000,6,1.0
...,...,...,...
2659843,67164000,4708,1.0
2659844,67164000,4710,1.0
2659845,67164000,4711,1.0
2659846,67164000,4712,1.0


In [60]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,0,3010,3014,3015,3016,3017,3019,3021,3022,...,3034,3011,3004,3126,3002,2968,2970,2973,2974,2976
1,59882000,0,3010,3014,3015,3016,3017,3019,3021,3022,...,3034,3011,3004,3126,3002,2968,2970,2973,2974,2976
2,59886000,0,3010,3014,3015,3016,3017,3019,3021,3022,...,3034,3011,3004,3126,3002,2968,2970,2973,2974,2976
3,59890000,0,3010,3014,3015,3016,3017,3019,3021,3022,...,3034,3011,3004,3126,3002,2968,2970,2973,2974,2976
4,59892000,0,3010,3014,3015,3016,3017,3019,3021,3022,...,3034,3011,3004,3126,3002,2968,2970,2973,2974,2976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,67149000,0,3010,3014,3015,3016,3017,3019,3021,3022,...,3034,3011,3004,3126,3002,2968,2970,2973,2974,2976
917,67154000,0,3010,3014,3015,3016,3017,3019,3021,3022,...,3034,3011,3004,3126,3002,2968,2970,2973,2974,2976
918,67160000,0,3010,3014,3015,3016,3017,3019,3021,3022,...,3034,3011,3004,3126,3002,2968,2970,2973,2974,2976
919,67161000,0,3010,3014,3015,3016,3017,3019,3021,3022,...,3034,3011,3004,3126,3002,2968,2970,2973,2974,2976


In [61]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59879000,"[0, 3010, 3014, 3015, 3016, 3017, 3019, 3021, ..."
1,59882000,"[0, 3010, 3014, 3015, 3016, 3017, 3019, 3021, ..."
2,59886000,"[0, 3010, 3014, 3015, 3016, 3017, 3019, 3021, ..."
3,59890000,"[0, 3010, 3014, 3015, 3016, 3017, 3019, 3021, ..."
4,59892000,"[0, 3010, 3014, 3015, 3016, 3017, 3019, 3021, ..."
...,...,...
916,67149000,"[0, 3010, 3014, 3015, 3016, 3017, 3019, 3021, ..."
917,67154000,"[0, 3010, 3014, 3015, 3016, 3017, 3019, 3021, ..."
918,67160000,"[0, 3010, 3014, 3015, 3016, 3017, 3019, 3021, ..."
919,67161000,"[0, 3010, 3014, 3015, 3016, 3017, 3019, 3021, ..."


In [62]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
# vod_list = pd.read_csv('../data/vod_list_add10.csv', index_col=0)
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59879000,"[0.0, 3010.0, 3014.0, 3015.0, 3016.0, 3017.0, ..."
1,59882000,"[0.0, 3010.0, 3014.0, 3015.0, 3016.0, 3017.0, ..."
2,59886000,"[0.0, 3010.0, 3014.0, 3015.0, 3016.0, 3017.0, ..."
3,59890000,"[0.0, 3010.0, 3014.0, 3015.0, 3016.0, 3017.0, ..."
4,59892000,"[0.0, 3010.0, 3014.0, 3015.0, 3016.0, 3017.0, ..."
...,...,...
916,67149000,"[0.0, 3010.0, 3014.0, 3015.0, 3016.0, 3017.0, ..."
917,67154000,"[0.0, 3010.0, 3014.0, 3015.0, 3016.0, 3017.0, ..."
918,67160000,"[0.0, 3010.0, 3014.0, 3015.0, 3016.0, 3017.0, ..."
919,67161000,"[0.0, 3010.0, 3014.0, 3015.0, 3016.0, 3017.0, ..."


In [ ]:
testdata = pd.read_csv('../data/watched_vod_10.csv', index_col=0)
testdata = testdata.groupby('subsr')['vod_id'].unique().reset_index()
testdata

In [63]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict)

0.0

### SVDpp
-

In [ ]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': np.arange(150, 1200, 5), 'lr_all': [0.01, 0.05],
              'reg_all': [0.05, 0.09], 'random_state' : [42]}
              # lr_all – The learning rate for all parameters. Default is 0.005.
              # reg_all – The regularization term for all parameters. Default is 0.02.

gs = GridSearchCV(SVD, param_grid, measures=['mae'], cv=5)

reader = Reader(rating_scale=(0,1))
train_set = Dataset.load_from_df(con_score[['subsr', 'vod_id', 'click_score']], reader)

gs.fit(train_set)
gs.best_params

In [ ]:
#
svdpp = SVDpp(n_epochs= 95, lr_all= 0.005, reg_all= 0.06, random_state= 42)
svdpp.fit(trainset)

In [ ]:
user_id = sorted(con_score.subsr.unique())
vod_id = sorted(con_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(svd.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,0.273517
1,59879000,1,0.099659
2,59879000,3,0.228638
3,59879000,5,0.190729
4,59879000,6,0.200687
...,...,...,...
2409315,67164000,4708,0.261200
2409316,67164000,4710,0.381770
2409317,67164000,4711,0.231248
2409318,67164000,4712,0.102031


In [ ]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,3838,1243,784,1863,979,2537,789,1191,498,...,112,2838,1921,4223,2288,3954,510,1214,1599,2226
1,59882000,3183,296,789,2537,2307,2288,3285,1243,112,...,1027,1227,240,882,2336,293,1599,868,1617,818
2,59886000,979,2576,137,112,1329,3183,308,555,1958,...,144,3891,3141,2302,2556,1233,1243,2667,734,388
3,59890000,2716,784,137,1329,3183,754,3779,3705,3838,...,296,538,1749,1877,448,4146,498,1050,868,2221
4,59892000,2867,1958,2537,868,137,1233,144,728,2288,...,2865,10,3029,2560,240,538,296,4672,445,3705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,67149000,1191,3141,868,1863,3196,1243,296,2537,1166,...,2290,581,2288,3891,1772,2865,3272,112,1709,1865
895,67154000,476,32,847,1401,4121,240,3838,1191,1877,...,3196,2875,549,2288,112,756,509,1107,538,784
896,67160000,4114,1214,112,2718,4446,821,32,2343,1227,...,316,2537,2109,2148,245,3768,610,296,2057,1786
897,67161000,1191,868,3824,458,4612,4101,4223,2226,1575,...,3705,4121,2949,433,3196,1943,818,320,2024,4469


In [ ]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59879000,"[3838, 1243, 784, 1863, 979, 2537, 789, 1191, ..."
1,59882000,"[3183, 296, 789, 2537, 2307, 2288, 3285, 1243,..."
2,59886000,"[979, 2576, 137, 112, 1329, 3183, 308, 555, 19..."
3,59890000,"[2716, 784, 137, 1329, 3183, 754, 3779, 3705, ..."
4,59892000,"[2867, 1958, 2537, 868, 137, 1233, 144, 728, 2..."
...,...,...
894,67149000,"[1191, 3141, 868, 1863, 3196, 1243, 296, 2537,..."
895,67154000,"[476, 32, 847, 1401, 4121, 240, 3838, 1191, 18..."
896,67160000,"[4114, 1214, 112, 2718, 4446, 821, 32, 2343, 1..."
897,67161000,"[1191, 868, 3824, 458, 4612, 4101, 4223, 2226,..."


In [ ]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
# vod_list = pd.read_csv('../data/vod_list_add10.csv', index_col=0)
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59879000,"[3838.0, 1243.0, 784.0, 1863.0, 979.0, 2537.0,..."
1,59882000,"[3183.0, 296.0, 789.0, 2537.0, 2307.0, 2288.0,..."
2,59886000,"[979.0, 2576.0, 137.0, 112.0, 1329.0, 3183.0, ..."
3,59890000,"[784.0, 137.0, 1329.0, 3183.0, 754.0, 3779.0, ..."
4,59892000,"[2867.0, 1958.0, 868.0, 137.0, 1233.0, 144.0, ..."
...,...,...
894,67149000,"[1191.0, 3141.0, 868.0, 1863.0, 3196.0, 1243.0..."
895,67154000,"[476.0, 32.0, 847.0, 1401.0, 4121.0, 240.0, 38..."
896,67160000,"[4114.0, 1214.0, 112.0, 2718.0, 4446.0, 821.0,..."
897,67161000,"[1191.0, 868.0, 458.0, 4612.0, 4101.0, 4223.0,..."


In [ ]:
testdata = pd.read_csv('../data/watched_vod_10.csv', index_col=0)
testdata = testdata.groupby('subsr')['vod_id'].unique().reset_index()
testdata

In [ ]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict)

0.012307692307692311

### BaselineOnly (0.012307692307692311)

In [ ]:
bsl_options = {'method' : 'als', 'n_epochs' : 60}
baseline = BaselineOnly(bsl_options=bsl_options)
baseline.fit(trainset)

Estimating biases using als...


In [ ]:
user_id = sorted(con_score.subsr.unique())
vod_id = sorted(con_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(baseline.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,0.129002
1,59879000,1,0.129262
2,59879000,3,0.184380
3,59879000,5,0.127621
4,59879000,6,0.151202
...,...,...,...
2409315,67164000,4708,0.145030
2409316,67164000,4710,0.145343
2409317,67164000,4711,0.145322
2409318,67164000,4712,0.145528


In [ ]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,2537,2288,2865,2057,296,2608,1166,538,143,...,401,2866,847,8,112,2058,4018,3838,336,256
1,59882000,2537,2288,2865,2057,296,2608,1166,538,143,...,401,2866,847,8,112,2058,4018,3838,336,256
2,59886000,2537,2288,2865,2057,296,2608,1166,538,143,...,401,2866,847,8,112,2058,4018,3838,336,256
3,59890000,2537,2288,2865,2057,296,2608,1166,538,143,...,401,2866,847,8,112,2058,4018,3838,336,256
4,59892000,2537,2288,2865,2057,296,2608,1166,538,143,...,401,2866,847,8,112,2058,4018,3838,336,256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,67149000,2537,2288,2865,2057,296,2608,1166,538,143,...,401,2866,847,8,112,2058,4018,3838,336,256
895,67154000,2537,2288,2865,2057,296,2608,1166,538,143,...,401,2866,847,8,112,2058,4018,3838,336,256
896,67160000,2537,2288,2865,2057,296,2608,1166,538,143,...,401,2866,847,8,112,2058,4018,3838,336,256
897,67161000,2537,2288,2865,2057,296,2608,1166,538,143,...,401,2866,847,8,112,2058,4018,3838,336,256


In [ ]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59879000,"[2537, 2288, 2865, 2057, 296, 2608, 1166, 538,..."
1,59882000,"[2537, 2288, 2865, 2057, 296, 2608, 1166, 538,..."
2,59886000,"[2537, 2288, 2865, 2057, 296, 2608, 1166, 538,..."
3,59890000,"[2537, 2288, 2865, 2057, 296, 2608, 1166, 538,..."
4,59892000,"[2537, 2288, 2865, 2057, 296, 2608, 1166, 538,..."
...,...,...
894,67149000,"[2537, 2288, 2865, 2057, 296, 2608, 1166, 538,..."
895,67154000,"[2537, 2288, 2865, 2057, 296, 2608, 1166, 538,..."
896,67160000,"[2537, 2288, 2865, 2057, 296, 2608, 1166, 538,..."
897,67161000,"[2537, 2288, 2865, 2057, 296, 2608, 1166, 538,..."


In [ ]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59879000,"[3838.0, 1243.0, 784.0, 1863.0, 979.0, 2537.0,..."
1,59882000,"[3183.0, 296.0, 789.0, 2537.0, 2307.0, 2288.0,..."
2,59886000,"[979.0, 2576.0, 137.0, 112.0, 1329.0, 3183.0, ..."
3,59890000,"[784.0, 137.0, 1329.0, 3183.0, 754.0, 3779.0, ..."
4,59892000,"[2867.0, 1958.0, 868.0, 137.0, 1233.0, 144.0, ..."
...,...,...
894,67149000,"[1191.0, 3141.0, 868.0, 1863.0, 3196.0, 1243.0..."
895,67154000,"[476.0, 32.0, 847.0, 1401.0, 4121.0, 240.0, 38..."
896,67160000,"[4114.0, 1214.0, 112.0, 2718.0, 4446.0, 821.0,..."
897,67161000,"[1191.0, 868.0, 458.0, 4612.0, 4101.0, 4223.0,..."


In [ ]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict, K = 10)

0.012307692307692311